In [200]:
import pandas as pd
import json

In [201]:
## thresholds 
attention_check_accuracy__threshold = 0.5

mean_accuracy__thresholds = {
    'ax_cpt': {'AX': 0.9, 'BX': 0.75, 'AY': 0.75, 'BY': 0.75},
    'cued_ts': {('switch', 'switch'): .85, ('switch', 'stay'): .9, ('stay', 'stay'): .9},
    'flanker': {'congruent': 0.9, 'incongruent': 0.75},
    'go_nogo': {'go': 0.9, 'nogo': 0.7},
    'n_back': {1.0: 0.8, 2.0: 0.7},
    'span': {'response': .5, 'processing': .5},
    'spatial_cueing': {'valid': 0.85, 'invalid': 0.75, 'nocue': 0.8, 'doublecue': 0.8},
    'spatial_ts': {'tswitch_cswitch': 0.85, 'tstay_cstay': 0.9, 'tstay_cswitch': .9},
    'stop_signal': 0.9,
    'stroop': {'congruent': 0.9, 'incongruent': 0.7},
    'visual_search':{
    ('conjunction', 8.0): 0.95,
    ('conjunction', 24.0): 0.85,
    ('feature', 8.0): 0.99,
    ('feature', 24.0): 0.95
    }
}

mean_rt__thresholds = {
    'ax_cpt': {'AX': 750, 'BX': 900, 'AY': 850, 'BY': 900},
    'cued_ts': {('switch', 'switch'): 900, ('switch', 'stay'): 700, ('stay', 'stay'): 700},
    'flanker': {'congruent': 600, 'incongruent': 800},
    'go_nogo': {'go': 700},
    'n_back': {1.0: 850, 2.0: 950},
    'span': {'response': 1000, 'processing': 1000},
    'spatial_cueing': {'valid': 650, 'invalid': 750, 'nocue': 700, 'doublecue': 700},
    'spatial_ts': {'tswitch_cswitch': 900, 'tstay_cstay': 700, 'tstay_cswitch': 700},
    'stop_signal': 650,
    'stroop': {'congruent': 650, 'incongruent': 900},
    'visual_search': {
    ('conjunction', 8.0): 1000,
    ('conjunction', 24.0): 1200,
    ('feature', 8.0): 600,
    ('feature', 24.0): 800
}
}

omission_rate__thresholds = {
    'ax_cpt': .1,
    'cued_ts': {('switch', 'switch'): .1, ('switch', 'stay'): .1, ('stay', 'stay'): .1},
    'flanker': 0.1,
    'go_nogo': {'go': 0.1},
    'n_back': {1.0: 0.1, 2.0: 0.15},
    'span': {'response': .2, 'processing': .2},
    'spatial_cueing': {'valid': 0.1, 'invalid': 0.1, 'nocue': 0.1, 'doublecue': 0.1},
    'spatial_ts': {'tswitch_cswitch': .1, 'tstay_cstay': .1, 'tstay_cswitch': .1},
    'stop_signal': 0.1,
    'stroop': 0.1,
    'visual_search': {
    ('conjunction', 8.0): 0.1,
    ('conjunction', 24.0): 0.15,
    ('feature', 8.0): 0.05,
    ('feature', 24.0): 0.1
}
}

# Define the stopping task thresholds
mean_stopping__thresholds = {
    'stop_signal': {
        'stop_acc': 0.7,  
        'go_acc': 0.7,    
        'avg_go_rt': 600,
        'stop_success': 0.5,  
        'stop_fail': 0.5,    
        'go_success': 0.5,   
        'stop_omission_rate': 0.2,  
        'go_omission_rate': 0.2    
    }
}

# Define the 'span' task thresholds outside of the function
mean_span__thresholds = {
    'accuracy': {
        'same-domain': 0.7, 
        'storage-only': 0.6, 
    },
    'rt': {
        'same-domain': 700, 
    },
    'omission': {
        'omission_rate_empty_response_trials': 0.2,  
        'omission_rate_incomplete_response_trials': 0.1,
        'omission_rate_processing_trials': 0.3, 
    }
}


In [202]:
def feedback_generator(task, accuracies=None, rts=None, omissions=None, attention_check_accuracy=None, factorial_condition=False, stopping_data=None):
    feedbacks = []

    def get_threshold(thresholds, condition, factorial_condition):
        if factorial_condition and isinstance(thresholds, dict):
            # For factorial conditions, we expect a dictionary with tuple keys
            return thresholds.get(condition)
        elif not factorial_condition and isinstance(thresholds, dict):
            # For non-factorial conditions, we extract the value using the condition as the key
            return thresholds.get(condition)
        else:
            # When thresholds is not a dictionary, it means it's a direct value applicable to all conditions
            return thresholds

    if accuracies:
        if task not in mean_accuracy__thresholds:
            return ["Invalid task name."]
        for condition, value in accuracies.items():
            threshold = mean_accuracy__thresholds[task]
            target = get_threshold(threshold, condition, factorial_condition)
            if target is None:
                # feedbacks.append(f"Invalid condition {condition} for the task {task}.")
                continue
            if value < target:
                condition_label = ' and '.join(condition) if factorial_condition else condition
                feedback = f"Your accuracy of {value*100:.2f}% is low for the {condition_label} condition of {task}."
                feedbacks.append(feedback)

    if omissions:
        if task not in omission_rate__thresholds:
            return ["Invalid task name."]
        for condition, value in omissions.items():
            threshold = omission_rate__thresholds[task]
            target = get_threshold(threshold, condition, factorial_condition)
            if target is None:
                # feedbacks.append(f"Invalid condition {condition} for the task {task}.")
                continue
            if value > target:
                condition_label = ' and '.join(condition) if factorial_condition else condition
                feedback = f"Your omission rate of {value*100:.2f}% is high for the {condition_label} condition of {task}."
                feedbacks.append(feedback)

    if rts:
        if task not in mean_rt__thresholds:
            return ["Invalid task name."]
        for condition, value in rts.items():
            threshold = mean_rt__thresholds[task]
            target = get_threshold(threshold, condition, factorial_condition)
            if target is None:
                # feedbacks.append(f"Invalid condition {condition} for the task {task}.")
                continue
            if value > target:
                condition_label = ' and '.join(condition) if factorial_condition else condition
                feedback = f"Your average RT of {value:.2f}ms is high for the {condition_label} condition of {task}."
                feedbacks.append(feedback)

    if attention_check_accuracy is not None:
        if attention_check_accuracy < attention_check_accuracy__threshold:
            feedback = f"Your attention check accuracy of {attention_check_accuracy*100:.2f}% is below the required threshold of {attention_check_accuracy__threshold*100:.2f}%. Please ensure you are remaining attentive throughout the tasks."
            feedbacks.append(feedback)

    if stopping_data and task == 'stop_signal':
        stopping_thresholds = mean_stopping__thresholds[task]
        for metric, value in stopping_data.items():
            threshold = stopping_thresholds.get(metric)
            if threshold is not None:
                if 'acc' in metric and value < threshold:
                    feedback = f"Your {metric.replace('_', ' ')} of {value*100:.2f}% is below the threshold of {threshold*100:.2f}%."
                    feedbacks.append(feedback)
                elif 'rt' in metric and value > threshold:
                    feedback = f"Your {metric.replace('_', ' ')} of {value:.2f}ms is above the threshold of {threshold:.2f}ms."
                    feedbacks.append(feedback)
                elif 'omission_rate' in metric and value > threshold:
                    feedback = f"Your {metric.replace('_', ' ')} of {value*100:.2f}% is above the threshold of {threshold*100:.2f}%."
                    feedbacks.append(feedback)

    # Section to handle 'span' task feedback
    if task == 'span':
        span_accuracy_thresholds = mean_span__thresholds['accuracy']
        span_rt_thresholds = mean_span__thresholds['rt']
        span_omission_thresholds = mean_span__thresholds['omission']
        
        # Process accuracies for 'span'
        if accuracies:
            for condition, value in accuracies.items():
                threshold = span_accuracy_thresholds.get(condition)
                if threshold is None:
                    # feedbacks.append(f"Invalid condition {condition} for the task {task}.")
                    continue
                if value < threshold:
                    feedback = f"Your accuracy for {condition} is low at {value*100:.2f}%, below the threshold of {threshold*100:.2f}%."
                    feedbacks.append(feedback)

        # Process RTs for 'span'
        if rts:
            for condition, value in rts.items():
                threshold = span_rt_thresholds.get(condition)
                if threshold is None:
                    # feedbacks.append(f"Invalid condition {condition} for the task {task}.")
                    continue
                if value > threshold:
                    feedback = f"Your reaction time for {condition} is high at {value:.2f}ms, above the threshold of {threshold:.2f}ms."
                    feedbacks.append(feedback)

        # Process omissions for 'span'
        if omissions:
            for metric, value in omissions.items():
                threshold = span_omission_thresholds.get(metric)
                if threshold is None:
                    feedbacks.append(f"Invalid metric {metric} for the task {task}.")
                    continue
                if value > threshold:
                    feedback = f"Your {metric.replace('_', ' ')} is high at {value*100:.2f}%, above the threshold of {threshold*100:.2f}%."
                    feedbacks.append(feedback)

    return feedbacks


In [203]:
def get_stopping_data(df, split_by_block_num=False):
    """
    Extracts and calculates metrics related to 'stop' and 'go' conditions for test trials.
    
    The function processes data to compute key metrics, including accuracy, response time (rt),
    stop signal delay (SSD), and omission rates, for both 'stop' and 'go' trial conditions.
    The results can be grouped by block numbers if required.
    
    Input:
      df: DataFrame containing task data for a specific task for a single subject.
      split_by_block_num (optional): Boolean flag to determine if results should be grouped 
      by block number (default is False).
      
    Output:
      Prints the computed metrics either grouped by block numbers or in an aggregated form.
      
    Metrics Calculated:
      - stop_acc: Mean accuracy for 'stop' trials.
      - go_acc: Mean accuracy for 'go' trials.
      - avg_go_rt: Average response time for correct 'go' trials.
      - max_SSD: Maximum stop signal delay.
      - min_SSD: Minimum stop signal delay.
      - mean_SSD: Average stop signal delay.
      - stop_success: Percentage of successful stops.
      - stop_fail: Percentage of failed stops.
      - go_success: Percentage of successful 'go' trials.
      - stop_omission_rate: Omission rate for 'stop' trials.
      - go_omission_rate: Omission rate for 'go' trials.
    """
    test_trials__df = df[(df['trial_id'] == 'test_trial')]
    
    grouping_column = 'block_num' if split_by_block_num else None

    # If we're splitting by block_num, group the data by block_num
    if split_by_block_num:
        stop_trials = test_trials__df[(test_trials__df['condition'] == 'stop')].groupby(grouping_column)
        go_trials = test_trials__df[(test_trials__df['condition'] == 'go')].groupby(grouping_column)
    else:
        stop_trials = test_trials__df[(test_trials__df['condition'] == 'stop')]
        go_trials = test_trials__df[(test_trials__df['condition'] == 'go')]

    # Define a helper function to calculate metrics for a given group
    def calculate_metrics(group):
        stop_acc = group[group['condition'] == 'stop']['stop_acc'].mean()
        go_acc = group[group['condition'] == 'go']['go_acc'].mean()

        go_correct_trials = group[(group['condition'] == 'go') & (group['go_acc'] == 1)]
        avg_go_rt = go_correct_trials['rt'].mean()

        max_SSD = group['SSD'].max()
        min_SSD = group['SSD'].min()
        mean_SSD = group['SSD'].mean()

        stop_success = group[group['condition'] == 'stop']['stop_acc'].mean()
        stop_fail = 1 - stop_success

        go_success = group[group['condition'] == 'go']['go_acc'].mean()
        stop_omission_rate = group[group['condition'] == 'stop']['rt'].isna().mean()
        go_omission_rate = group[group['condition'] == 'go']['rt'].isna().mean()

        return {
            "stop_acc": stop_acc,
            "go_acc": go_acc,
            "avg_go_rt": avg_go_rt,
            "max_SSD": max_SSD,
            "min_SSD": min_SSD,
            "mean_SSD": mean_SSD,
            "stop_success": stop_success,
            "stop_fail": stop_fail,
            "go_success": go_success,
            "stop_omission_rate": stop_omission_rate,
            "go_omission_rate": go_omission_rate
        }

    # If we're splitting by block_num, apply the helper function to each group
    if split_by_block_num:
        results = test_trials__df.groupby(grouping_column).apply(calculate_metrics)
    else:
        results = calculate_metrics(test_trials__df)
    return results

In [204]:
def calculate_attention_check_accuracy(df):
    """
    Calculates the attention check accuracy for attention checks
    
    This function computes the accuracy for a given set of attention checks for a single task df.
    
    Input:
      df: DataFrame containing task data for a specific task for a single subject.  
          
    Output:
      Prints the overall attention check accuracy for a given task df for a single subject. 
    """

    test_trials__df = df[(df['trial_id'] == 'test_attention_check')]
    attention_check_accuracy = test_trials__df['correct_trial'].mean()
    return attention_check_accuracy

In [205]:
def calculate_average_rt(df, condition_col='condition', test_trial='test_trial', correct_trial_col='correct_trial', factorial_condition=False,factorial_conditions=[], split_by_block_num=False):
    """
    Calculates the average reaction time (RT) for given test trials based on specific conditions.
    
    This function can handle both standard conditions and factorial conditions. Additionally,
    results can optionally be split by block number.
    
    Input:
      df: DataFrame containing the task data.
      condition_col: Name of the column representing the condition. Default is 'condition'.
      test_trial: Name of the column indicating the type of trial. Default is 'test_trial'.
      correct_trial_col: Column indicating if the trial was correctly executed. Default is 'correct_trial'.
      factorial_condition: Boolean to specify if the data has factorial conditions. Default is False.
      factorial_conditions: List of columns indicating factorial conditions. Default is [].
      split_by_block_num: Boolean to specify if results should be split by block number. Default is False.
    
    Output:
      Prints the average RT for the specified conditions.
    """    
    test_trials__df = df[(df['trial_id'] == test_trial) & (df[correct_trial_col] == 1)]
    
    if factorial_condition:
        grouping_columns = factorial_conditions
    else:
        grouping_columns = [condition_col]
    
    if split_by_block_num:
        grouping_columns.append('block_num')
    
    rt_by_condition = test_trials__df.groupby(grouping_columns).apply(lambda group: group['rt'].mean())
    
    return rt_by_condition.to_dict()

In [206]:
def calculate_omission_rate(df, test_trial='test_trial', condition_col='condition', factorial_condition=False, factorial_conditions=[], split_by_block_num=False, is_go_no_go=False):
    """
    Calculates the omission rate for given test trials based on specific conditions.
    
    Omission rate refers to the proportion of missing reaction times (RTs) in the data. This function
    supports calculations for both standard and factorial conditions. Results can optionally be split 
    by block number.
    
    Input:
       df: DataFrame containing task data for a specific task for a single subject.
      test_trial: Name of the column indicating the type of trial. Default is 'test_trial'.
      condition_col: Name of the column representing the condition. Default is 'condition'.
      factorial_condition: Boolean to specify if the data has factorial conditions. Default is False.
      factorial_conditions: List of columns indicating factorial conditions. Default is [].
      split_by_block_num: Boolean to specify if results should be split by block number. Default is False.
    
    Output:
      Prints the omission rate for the specified conditions.
    """
     
    test_trials__df = df[df['trial_id'] == test_trial]
    
    if factorial_condition:
        grouping_columns = factorial_conditions
    else:
        grouping_columns = [condition_col]
    
    if split_by_block_num:
        grouping_columns.append('block_num')
    
    omission_rate = test_trials__df.groupby(grouping_columns).apply(lambda group: group['rt'].isna().mean())

    if is_go_no_go:
        omission_rate = omission_rate['go']

    return omission_rate.to_dict()

In [207]:
def calculate_omission_rate__span(df):
    """
    Calculates the omission rate for the 'span' task based on specific trial types and response lengths.
    
    This function targets the 'span' task data to calculate two types of omissions: 
    1) Completely empty responses, and 
    2) Incomplete responses (i.e., responses with a length between 1 and 3).
    
    Additionally, this function calculates the omission rate for 'test_inter-stimulus' trials, 
    which is the proportion of missing reaction times (RTs) in these trials.

    Input:
       df: DataFrame containing task data for a specific task for a single subject.
    
    Output:
      Prints the mean number of empty responses, the mean number of incomplete responses, 
      and the omission rate for 'test_inter-stimulus' trials.
    """
   
    test_response_trials__df = df[df['trial_id'] == 'test_response'].copy()
    test_processing_trials__df = df[df['trial_id'] == 'test_inter-stimulus'].copy()

    # Convert the strings in the 'response' column to actual lists
    test_response_trials__df['response'] = test_response_trials__df['response']

    omission_rate_processing_trials = test_processing_trials__df['rt'].isna().mean()

    # Calculate the number of empty and incomplete responses
    test_response_trials__df['empty'] = test_response_trials__df['response'].apply(lambda x: len(x) == 0)
    test_response_trials__df['incomplete'] = test_response_trials__df['response'].apply(lambda x: 0 < len(x) < 4)

    # Get the mean of each type
    mean_empty = test_response_trials__df['empty'].mean()
    mean_incomplete = test_response_trials__df['incomplete'].mean()

    # Return the results as a dictionary
    return {
        'omission_rate_empty_response_trials': mean_empty,
        'omission_rate_incomplete_response_trials': mean_incomplete,
        'omission_rate_processing_trials': omission_rate_processing_trials
    }

In [208]:
def calculate_average_accuracy(df, correct_trial_col='correct_trial', condition_col='condition', test_trial='test_trial', factorial_condition=False, factorial_conditions=[], split_by_block_num=False):
    """
    Calculates the average accuracy for given test trials based on specified conditions.
    
    This function computes the mean accuracy for a given set of test trials. It allows for grouping
    by a single condition or multiple factorial conditions. The option to further split by block number
    is also available. The accuracy is determined by averaging the values in the `correct_trial_col`.
    
    Input:
      df: DataFrame containing task data for a specific task for a single subject.
      correct_trial_col (optional): Name of the column indicating correct trials (default is 'correct_trial').
      condition_col (optional): Name of the main condition column for grouping (default is 'condition').
      test_trial (optional): Specifies the trial type to be considered for accuracy calculation (default is 'test_trial').
      factorial_condition (optional): Boolean flag indicating if factorial conditions should be used for grouping (default is False).
      factorial_conditions (optional): List of columns to be used for factorial grouping (default is an empty list).
      split_by_block_num (optional): Boolean flag to determine if results should be split by block number (default is False).
      
    Output:
      Prints the average accuracy grouped by the specified conditions.
    """
   
    test_trials__df = df[df['trial_id'] == test_trial]
    
    if factorial_condition:
        grouping_columns = factorial_conditions
    else:
        grouping_columns = [condition_col]
    
    if split_by_block_num:
        grouping_columns.append('block_num')
    
    accuracy_by_condition = test_trials__df.groupby(grouping_columns)[correct_trial_col].mean()
    
    return accuracy_by_condition.to_dict()


In [209]:
ax_cpt__json = './data/ax_cpt_rdoc_23-10-25-16:51.json'
cued_ts__json = './data/cued_task_switching_rdoc_23-10-25-17:36.json'
flanker__json = './data/flanker_rdoc_23-10-25-18:02.json'
go_nogo__json = './data/go_nogo_rdoc_23-10-25-18:20.json'
n_back__json = './data/n_back_rdoc_23-10-25-18:33.json' 
span__json = './data/span_rdoc__behavioral_23-10-25-21:45.json' 
spatial_ts__json = './data/spatial_task_switching_rdoc_23-10-25-20:33.json'
spatial_cueing__json = './data/spatial_cueing_rdoc_23-10-25-20:56.json'
stroop__json = './data/stroop_rdoc_23-10-25-18:44.json'
stop_signal__json = './data/stop_signal_rdoc_23-10-25-19:22.json'
visual_search__json = './data/visual_search_rdoc_23-10-25-19:10.json'

single_subject__json = {
    'ax_cpt': ax_cpt__json,
    'cued_ts': cued_ts__json,
    'flanker': flanker__json,
    'go_nogo': go_nogo__json,
    'n_back': n_back__json,
    'span': span__json,
    'spatial_ts': spatial_ts__json,
    'spatial_cueing': spatial_cueing__json,
    'stroop': stroop__json,
    'stop_signal': stop_signal__json,
    'visual_search': visual_search__json
}

In [210]:
single_subject__df = {}

for task_name in single_subject__json:
    with open(single_subject__json[task_name], 'r', encoding='utf-8') as file:
        data = json.load(file)
        trial_data = json.loads(data['trialdata'])
        df = pd.DataFrame(trial_data)
        single_subject__df[task_name] = df

In [211]:
feedback__all_tasks = {}

for task__name in single_subject__df:
    task__df = single_subject__df[task__name]
    if task__name == 'ax_cpt':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        omissions = calculate_omission_rate(task__df, test_trial='test_probe')
        accuracies = calculate_average_accuracy(task__df, test_trial='test_probe')
        rts = calculate_average_rt(task__df, test_trial='test_probe')
        feedbacks = feedback_generator('ax_cpt',  accuracies=accuracies, rts=rts, omissions=omissions,attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'cued_ts':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        accuracies = calculate_average_accuracy(task__df, factorial_condition=True, factorial_conditions=['cue_condition', 'task_condition']) # need to use factorial for cue_condition and task_condition since separate cols
        rts = calculate_average_rt(task__df, factorial_condition=True, factorial_conditions=['cue_condition', 'task_condition'])
        omissions = calculate_omission_rate(task__df, factorial_condition=True, factorial_conditions=['cue_condition', 'task_condition'])
        feedbacks = feedback_generator('cued_ts',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'flanker':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        omissions = calculate_omission_rate(task__df)
        accuracies = calculate_average_accuracy(task__df)
        rts = calculate_average_rt(task__df)
        feedbacks = feedback_generator('flanker',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'go_nogo':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        omissions = calculate_omission_rate(task__df)
        accuracies = calculate_average_accuracy(task__df)
        rts = calculate_average_rt(task__df)
        feedbacks = feedback_generator('go_nogo',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'n_back':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        accuracies = calculate_average_accuracy(task__df, condition_col='delay') # need to use delay instead of 'match' , 'mismatch' condition 
        rts = calculate_average_rt(task__df, condition_col='delay') # need to use delay instead of 'match' , 'mismatch' condition 
        omissions = calculate_omission_rate(task__df, condition_col='delay')
        feedbacks = feedback_generator('n_back',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'span':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        accuracies = calculate_average_accuracy(task__df, test_trial='test_response')
        rts = calculate_average_rt(task__df, test_trial='test_inter-stimulus', correct_trial_col='correct_response')
        omissions = calculate_omission_rate__span(task__df) # need something different for omissions since no response in test_response is [] and incomplete is [].length < 4
        feedbacks = feedback_generator('span', accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'spatial_ts':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        omissions = calculate_omission_rate(task__df)
        accuracies = calculate_average_accuracy(task__df)
        rts = calculate_average_rt(task__df)
        feedbacks = feedback_generator('spatial_ts',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'spatial_cueing':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        omissions = calculate_omission_rate(task__df)
        accuracies = calculate_average_accuracy(task__df)
        rts = calculate_average_rt(task__df)
        feedbacks = feedback_generator('spatial_cueing',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'stroop':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        omissions = calculate_omission_rate(task__df)
        accuracies = calculate_average_accuracy(task__df)
        rts = calculate_average_rt(task__df)
        feedbacks = feedback_generator('stroop',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'stop_signal':
        attention_check_accuracy = calculate_attention_check_accuracy(task__df)
        stopping_data = get_stopping_data(task__df)
        feedbacks = feedback_generator('stop_signal', stopping_data=stopping_data, attention_check_accuracy=attention_check_accuracy)
    elif task__name == 'visual_search':
        attention_check_accuracy=attention_check_accuracy
        accuracies = calculate_average_accuracy(task__df, factorial_condition=True, factorial_conditions=['condition', 'num_stimuli']) # need to use factorial for load and feature/conjunction
        rts = calculate_average_rt(task__df, factorial_condition=True, factorial_conditions=['condition', 'num_stimuli'])
        omissions = calculate_omission_rate(task__df, factorial_condition=True, factorial_conditions=['condition', 'num_stimuli'])
        feedbacks = feedback_generator('visual_search',  accuracies=accuracies, rts=rts, omissions=omissions, attention_check_accuracy=attention_check_accuracy)
    feedback__all_tasks[task__name] = feedbacks

In [212]:
feedback__all_tasks

{'ax_cpt': ['Your accuracy of 33.33% is low for the AX condition of ax_cpt.',
  'Your accuracy of 40.00% is low for the AY condition of ax_cpt.',
  'Your accuracy of 53.33% is low for the BX condition of ax_cpt.',
  'Your accuracy of 40.00% is low for the BY condition of ax_cpt.',
  'Your omission rate of 53.33% is high for the AX condition of ax_cpt.',
  'Your omission rate of 56.67% is high for the AY condition of ax_cpt.',
  'Your omission rate of 36.67% is high for the BX condition of ax_cpt.',
  'Your omission rate of 56.67% is high for the BY condition of ax_cpt.'],
 'cued_ts': ["Your accuracy of 70.83% is low for the ('stay', 'stay') condition of cued_ts.",
  "Your accuracy of 54.17% is low for the ('switch', 'stay') condition of cued_ts.",
  "Your accuracy of 60.42% is low for the ('switch', 'switch') condition of cued_ts.",
  "Your omission rate of 20.83% is high for the ('stay', 'stay') condition of cued_ts.",
  "Your omission rate of 31.25% is high for the ('switch', 'stay')